<a href="https://colab.research.google.com/github/javiagu13/DeepLearning/blob/main/lab9_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 9
In this lab you will work through transfer learning using the Inception (version3) network for classifying cats and dogs.

---

### A. Transfer Learning
Model definition: the code cell below does the following:

1. Downloads the weights for a pre-trained inception network, and then instantiates a new instance of it using those weights.
2. Selects one of the pre-trained convolutional layers as the last layer for our new model to take as input.

In [ ]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

# 1.
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, # Whether to include the fully-connected layer at the top, as the last layer of the network. Default True
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

# Freeze the layers
for layer in pre_trained_model.layers:
  layer.trainable = False

# Uncomment the code below to see the (huge) model summary  
# pre_trained_model.summary()

# 2. 
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output


### Adding layers and compiling model
3. Add a fully connected layer to the pre-trained layers that have been defined as input to our model

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# 3.1 Flatten the output layer to 1 dimension

# 3.2 Add a fully connected layer with 1,024 hidden units and ReLU activation

# For Part C: (Do this after completing 9) 10. Add a dropout rate of 0.2

# 3.3 Add a final sigmoid layer for classification

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])


### Data preparation
4. Download cats vs dogs data

In [ ]:
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile

local_zip = '//tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

# For Part B: (Do this after completing 6) 7. Add data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1/255) 

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1/255 )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

### Training
5. Train the model consisting of pre-trained layers from Inception v3 and new layers defined in 3.

In [ ]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 2)

### Model performance
6, 9, 11. Plot the graph of the training and validation accuracies

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()